In [6]:
import csv
import pandas as pd
from datetime import timedelta

input_file = 'Conso.csv'
backup_file = 'Conso_backup.csv'
hours_to_add = 2

# === 1) Détection du séparateur ===
with open(input_file, 'r', encoding='utf-8', errors='replace') as f:
    sample = ''.join([line for line in f.readlines() if line.strip()][:10])  # éviter lignes vides
sniffer = csv.Sniffer()
try:
    dialect = sniffer.sniff(sample)
    delim = dialect.delimiter
except Exception:
    delim = ';'  # valeur par défaut
print(f"✅ Séparateur détecté : {repr(delim)}")

# === 2) Lecture du CSV (en texte brut pour ne rien perdre) ===
try:
    df = pd.read_csv(
        input_file,
        sep=delim,
        dtype=str,
        keep_default_na=False,
        na_values=[],
        engine='python',
        encoding='utf-8'
    )
except UnicodeDecodeError:
    df = pd.read_csv(
        input_file,
        sep=delim,
        dtype=str,
        keep_default_na=False,
        na_values=[],
        engine='python',
        encoding='ISO-8859-1'
    )
print(f"✅ Fichier '{input_file}' chargé ({len(df)} lignes, {len(df.columns)} colonnes)")

# === 3) Fonction de décalage date/heure ===
def shift_date_time(df, col_date, col_time, hours=2):
    if col_date not in df.columns or col_time not in df.columns:
        print(f"⚠️ Colonnes non trouvées : {col_date}, {col_time} — ignorées.")
        return 0

    combined = df[col_date].astype(str).str.strip() + ' ' + df[col_time].astype(str).str.strip()
    dt = pd.to_datetime(combined, dayfirst=True, format='%d/%m/%y %H:%M:%S', errors='coerce')

    if dt.notna().sum() == 0:
        dt = pd.to_datetime(combined, dayfirst=True, errors='coerce')

    valid = dt.notna().sum()
    if valid == 0:
        print(f"⚠️ Aucune date valide trouvée dans {col_date}/{col_time}.")
        return 0

    # Décalage horaire
    dt = dt + timedelta(hours=hours)

    df.loc[dt.notna(), col_date] = dt.dt.strftime('%d/%m/%y')
    df.loc[dt.notna(), col_time] = dt.dt.strftime('%H:%M:%S')

    print(f"✅ Décalage de {hours}h appliqué sur {col_date}/{col_time} ({valid} lignes).")
    return valid

# === 4) Sauvegarde de sécurité ===
df.to_csv(backup_file, sep=delim, index=False, encoding='utf-8-sig')
print(f"💾 Sauvegarde créée : {backup_file}")

# === 5) Application du décalage sur RTC et GPS ===
shift_date_time(df, 'RTC date', 'RTC time', hours=hours_to_add)
shift_date_time(df, 'GPS date', 'GPS time', hours=hours_to_add)

# === 6) Sauvegarde du fichier modifié ===
df.to_csv(input_file, sep=delim, index=False, encoding='utf-8-sig')
print(f"✅ Fichier final mis à jour : {input_file}")


✅ Séparateur détecté : ';'
✅ Fichier 'Conso.csv' chargé (164397 lignes, 12 colonnes)
💾 Sauvegarde créée : Conso_backup.csv
✅ Décalage de 2h appliqué sur RTC date/RTC time (164397 lignes).
✅ Décalage de 2h appliqué sur GPS date/GPS time (140676 lignes).
✅ Fichier final mis à jour : Conso.csv


In [9]:
import pandas as pd
import numpy as np

# === Chargement rapide des fichiers CSV ===
assistance_df = pd.read_csv('Assistance.csv', sep=';', dtype=str, encoding='utf-8', low_memory=False)
conso_df = pd.read_csv('Conso.csv', sep=';', dtype=str, encoding='utf-8', low_memory=False)

# === Conversion des dates/temps Conso en un seul datetime (plus rapide qu'à chaque boucle) ===
conso_df['RTC datetime'] = pd.to_datetime(
    conso_df['RTC date'].astype(str).str.strip() + ' ' + conso_df['RTC time'].astype(str).str.strip(),
    format='%d/%m/%y %H:%M:%S',
    errors='coerce'
)

# Conversion des colonnes moteur en numérique, tolérante
for col in ['Motor 1', 'Motor 2']:
    if col in conso_df.columns:
        conso_df[col] = (
            conso_df[col]
            .astype(str)
            .str.replace(',', '.', regex=False)
            .replace('', np.nan)
            .astype(float)
        )
    else:
        conso_df[col] = 0.0

# Préparation des colonnes de sortie
if 'Start - On Site' not in assistance_df.columns:
    assistance_df['Start - On Site'] = np.nan
if 'Lignes' not in assistance_df.columns:
    assistance_df['Lignes'] = ""

# === Conversion des colonnes de temps dans Assistance ===
assistance_df['Start Mobilisation'] = pd.to_datetime(
    assistance_df['Start Mobilisation'], format='%d/%m/%y %H:%M:%S', errors='coerce'
)
assistance_df['On Site'] = pd.to_datetime(
    assistance_df['On Site'], format='%d/%m/%y %H:%M:%S', errors='coerce'
)

# === Pré-tri du DataFrame conso pour accélérer les recherches ===
conso_df = conso_df.sort_values('RTC datetime').reset_index()
rtc_times = conso_df['RTC datetime'].values

# === Boucle vectorisée (rapide) sur les lignes d'Assistance ===
for i, row in assistance_df.iterrows():
    start_time = row['Start Mobilisation']
    end_time = row['On Site']

    if pd.notna(start_time) and pd.notna(end_time):
        # Trouver les indices des lignes de Conso dans la plage
        mask = (rtc_times >= start_time) & (rtc_times <= end_time)
        if mask.any():
            subset = conso_df.loc[mask, ['Motor 1', 'Motor 2']]
            subset = subset[(subset['Motor 1'] > 0) | (subset['Motor 2'] > 0)]
            total = subset['Motor 1'].sum(skipna=True) + subset['Motor 2'].sum(skipna=True)

            if total > 0:
                assistance_df.at[i, 'Start - On Site'] = total
                lignes = conso_df.loc[mask].index
                assistance_df.at[i, 'Lignes'] = f"{lignes.min()} - {lignes.max()}"

# === Sauvegarde du fichier ===
assistance_df.to_csv('Assistance_modifie.csv', sep=';', index=False, encoding='utf-8-sig')

print("✅ Calcul terminé — fichier 'Assistance_modifie.csv' généré avec succès.")


✅ Calcul terminé — fichier 'Assistance_modifie.csv' généré avec succès.
